## Optimising Model Parameters

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5




In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.300847  [   64/60000]
loss: 2.286402  [ 6464/60000]
loss: 2.267652  [12864/60000]
loss: 2.263565  [19264/60000]
loss: 2.247751  [25664/60000]
loss: 2.226422  [32064/60000]
loss: 2.225475  [38464/60000]
loss: 2.197228  [44864/60000]
loss: 2.188478  [51264/60000]
loss: 2.163023  [57664/60000]
Test Error: 
 Accuracy: 55.1%, Avg loss: 2.149874 

Epoch 2
-------------------------------
loss: 2.154812  [   64/60000]
loss: 2.146297  [ 6464/60000]
loss: 2.086319  [12864/60000]
loss: 2.105075  [19264/60000]
loss: 2.052777  [25664/60000]
loss: 1.993913  [32064/60000]
loss: 2.019059  [38464/60000]
loss: 1.938411  [44864/60000]
loss: 1.935554  [51264/60000]
loss: 1.873772  [57664/60000]
Test Error: 
 Accuracy: 60.3%, Avg loss: 1.862895 

Epoch 3
-------------------------------
loss: 1.890822  [   64/60000]
loss: 1.866744  [ 6464/60000]
loss: 1.740769  [12864/60000]
loss: 1.786032  [19264/60000]
loss: 1.684649  [25664/60000]
loss: 1.629415  [32064/600